In [1]:
!pip install -q ../input/for-pydicom/pylibjpeg-1.4.0-py3-none-any.whl
!pip install -q ../input/for-pydicom/python_gdcm-3.0.14-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q ../input/for-pydicom/pylibjpeg_libjpeg-1.3.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q -U keras-tuner

# Note : #

### In this notebook we are going to use Keras tuner to tune hyperparamteres, like Learning Rate to see improvement in the training process ###

In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from glob import glob
import pydicom
import tensorflow as tf
import tqdm as tqdm
import tensorflow_io as tfio
import pathlib
import datetime
from tensorflow import keras
import keras_tuner as kt

In [27]:
DATA_DIR = "../input/rsna-2022-cervical-spine-fracture-detection/"
trainCsv = "../input/rsna-2022-cervical-spine-fracture-detection/train.csv"
train_df = pd.read_csv(trainCsv)
print(train_df.head())

            StudyInstanceUID  patient_overall  C1  C2  C3  C4  C5  C6  C7
0   1.2.826.0.1.3680043.6200                1   1   1   0   0   0   0   0
1  1.2.826.0.1.3680043.27262                1   0   1   0   0   0   0   0
2  1.2.826.0.1.3680043.21561                1   0   1   0   0   0   0   0
3  1.2.826.0.1.3680043.12351                0   0   0   0   0   0   0   0
4   1.2.826.0.1.3680043.1363                1   0   0   0   0   1   0   0


In [28]:
def load_dicom(path):
    """
    reads a dicom file and loads the image array inside it
    inputs:
        path: the path of the required dicom file
    returns:
        data: image pixel arrays
    """
    img=pydicom.dcmread(path)
    data=img.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data

In [29]:
def data_generator():
    for i, study_instance in enumerate(train_df.StudyInstanceUID[:]):
        for dcm in os.listdir(DATA_DIR + f"train_images/{study_instance}"):
            train_labels = []
            path = DATA_DIR + f"train_images/{study_instance}/{dcm}"
            img = load_dicom(path)
            
            # resize each image into a shape of (512, 512)
            img = np.resize(img, (128, 128))
            #  normalize image
            img = img / 255.0
            # convert from gray scale to rgb, this will be helpful incase we want to use pretrained models
            img = tf.expand_dims(img, axis=-1)
            img = tf.image.grayscale_to_rgb(img)
            
            train_labels.extend([
                train_df.loc[i, "C1"],
                train_df.loc[i, "C2"],
                train_df.loc[i, "C3"],
                train_df.loc[i, "C4"],
                train_df.loc[i, "C5"],
                train_df.loc[i, "C6"],
                train_df.loc[i, "C7"],
                train_df.loc[i, "patient_overall"] # end with patient overall
            ])
            yield img, train_labels

In [30]:
# creating train, validation and test split for evaluation
def splitDataset(dataset, trainFactor, img_count): # here it refers to tf.dataset
    
    train_size = int(trainFactor * img_count)
    validation_size = int(((1 - trainFactor) / 2)* img_count)
    test_size = int(((1 - trainFactor) / 2)* img_count)
    
    train_dataset = dataset.take(train_size)
    test_dataset = dataset.skip(train_size)
    validation_dataset = test_dataset.skip(validation_size)
    test_dataset = test_dataset.take(test_size)
    
    """
    valid_test_split = (1 - trainFactor) / 2
    validation_dataset = dataset.take(int(valid_test_split* img_count))
    test_dataset = dataset.take(int(valid_test_split* img_count))
    """
    return train_dataset, validation_dataset, test_dataset

In [31]:
def configure_for_performance(data):
    data = data.cache()
    data = data.batch(16)
    data = data.prefetch(buffer_size=tf.data.AUTOTUNE)
    return data

## Writing my Own Custom Loss function based on the Evaluation metric ##
https://www.kaggle.com/code/andradaolteanu/rsna-fracture-detect-pytorch-densenet-train#1.-Evaluation-Metric-Understanding

https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/discussion/340612

## First Creating an Eample ##

In [32]:
competition_weights = {
    '-' : tf.constant([7, 1, 1, 1, 1, 1, 1, 1], dtype=tf.float32),
    '+' : tf.constant([14, 2, 2, 2, 2, 2, 2, 2], dtype=tf.float32),
}

print(competition_weights['-'])

tf.Tensor([7. 1. 1. 1. 1. 1. 1. 1.], shape=(8,), dtype=float32)


In [ ]:
import math
# Custom colors
class clr:
    S = '\033[1m' + '\033[94m'
    E = '\033[0m'
# Example

# Prediction (very bad)
logits = tf.constant([[0.2221, 0.1037, 0.0739, 0.1112, 0.1026, 0.0902, 0.1597, 0.1365],
                       [0.1702, 0.0952, 0.0815, 0.1262, 0.1185, 0.1097, 0.1675, 0.1312]])
print(clr.S+"Prediction:"+clr.E, "\n", logits)

# Actual
targets = tf.constant([[0., 0., 0., 0., 0., 0., 0., 0.],
                        [1., 0., 0., 0., 0., 0., 0., 1.]])
print("[*] Targtes Shape : ",targets.shape)

print(clr.S+"Target:"+clr.E, "\n", targets)

# Compute the weights
weights = targets * competition_weights['+'] + (1 - targets) * competition_weights['-']
print(clr.S+"Weights:"+clr.E, "\n", weights)

print("[*] Weights type : ", type(weights))
# Compute losses on label and exam level
L = np.zeros(targets.shape)

w = weights
y = targets
p = logits

for i in range(L.shape[0]):
    for j in range(L.shape[1]):
        L[i, j] = -w[i, j] * (y[i, j] * math.log(p[i, j]) +(1 - y[i, j]) * math.log(1 - p[i, j]))

L = tf.convert_to_tensor(L)
print(clr.S+"LOSSES:"+clr.E, "\n", L)

# Average Loss on Exam (or patient)
Exams_Loss = tf.math.divide(tf.cast(tf.reduce_sum(L, 1), dtype =tf.float32), tf.cast(tf.reduce_sum(w , 1), dtype = tf.float32))

print(clr.S+"Exam Losses:"+clr.E, "\n", Exams_Loss)

## Now writing the Loss function ##

In [33]:
def get_custom_loss_wrapper(input_tensor):
    def get_custom_loss(targets, logits):

        print("Writing loss function")
        # Compute the weights
        targets = tf.cast(targets, dtype = tf.float32)
        print("Targets : ", targets.shape)
        logits = tf.cast(logits, dtype = tf.float32)
        print("Logits : ", logits.shape)
        weights = targets * competition_weights['+'] + (1 - targets) * competition_weights['-']

        # Losses on label and exam level
        L = np.zeros(targets.shape)

        w = weights
        y = targets
        p = logits
        eps=1e-8

        for i in range(L.shape[0]):
            for j in range(L.shape[1]):
                L[i, j] = -w[i, j] * (y[i, j] * math.log(p[i, j] + eps) + (1 - y[i, j]) * math.log(1 - p[i, j] + eps))

        # Average Loss on Exam (or patient)
        Exams_Loss = tf.math.divide(tf.cast(tf.reduce_sum(L, 1), dtype =tf.float32), tf.cast(tf.reduce_sum(w , 1), dtype = tf.float32))
        return Exams_Loss

In [9]:
def creating_model(hp):
    IMG_SHAPE = (128, 128, 3)
    base_model = tf.keras.applications.EfficientNetB5(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
    
    base_model.trainable = False
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = base_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(8, activation = 'sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), 
              loss=get_custom_loss,
              metrics=[tf.keras.metrics.BinaryAccuracy()])
    
    return model

## Model without Keras Tuner ##

In [42]:
def creating_model_without_tuner():
    IMG_SHAPE = (128, 128, 3)
    base_model = tf.keras.applications.vgg16.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
    
    base_model.trainable = False
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = base_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(8, activation = 'sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)
    
    #hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss=get_custom_loss_wrapper(inputs),
              metrics=[tf.keras.metrics.BinaryAccuracy()])
    
    return model

In [35]:
# this is required because of length of dataset is not valid for generators
def getImageCount():
    img_count = 0
    for _, study_instance in enumerate(train_df.StudyInstanceUID[:5]):
        for _ in os.listdir(DATA_DIR + f"train_images/{study_instance}"):
            img_count += 1
            
    return img_count

In [36]:
def createTensorboardCallback(logdir):
    return tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)

def saveModelCallback(filePath):
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filePath, monitor='val_loss',verbose=1, save_best_only=True, mode='min')
    
    return checkpoint

In [37]:
def display(history):
    plt.plot(history.history['binary_accuracy'])
    plt.plot(history.history['val_binary_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [38]:
# getting the dataset
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# creating the dataset
dataset = tf.data.Dataset.from_generator(data_generator, (tf.float32, tf.int8))

# printing a sample data for checking
for img, label in dataset.take(1):
    print(img.shape)
    print(label.shape)
    print(label)

# splitting the dataset
trainFactor = 0.8
img_count = getImageCount()
print("[***] Images for training and validation : ", img_count)
train_data, validation_data, test_dataset = splitDataset(dataset, trainFactor, img_count)
#
train_dataset = configure_for_performance(train_data)
validation_dataset = configure_for_performance(validation_data)

# printing dataset structure after batching
print("[*] Dataset after batching")
for img, label in train_dataset.take(1):
    print(img.shape)
    print(label.shape)

(128, 128, 3)
(8,)
tf.Tensor([1 1 0 0 0 0 0 1], shape=(8,), dtype=int8)
[***] Images for training and validation :  1734
[*] Dataset after batching
(16, 128, 128, 3)
(16, 8)


2022-09-16 23:11:45.541429: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


## Using Keras tuner to tune the learning rate and see improvements ##

In [16]:
mydir = "/kaggle/working/my_dir"
os.makedirs(mydir, exist_ok=True)

tuner = kt.Hyperband(creating_model,
                     objective='val_binary_accuracy',
                     max_epochs=50,
                     factor=3,
                     directory=mydir,
                     project_name='rsna_baseline_improve_1')

# creating a stop early callback
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(train_dataset, epochs=10, validation_data = validation_dataset, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}""")

print("[*] Results Summary")
print(tuner.results_summary())


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
0.0001            |?                 |learning_rate
2                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
3                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/2
Writing loss function
Targets :  Tensor("get_custom_loss/Cast:0", dtype=float32)
Logits :  Tensor("get_custom_loss/cond/Identity_1:0", shape=(None, 8), dtype=float32)


TypeError: in user code:

    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /tmp/ipykernel_17/2221240378.py:12 get_custom_loss  *
        L = np.zeros(targets.shape)

    TypeError: expected sequence object with len >= 0 or a single integer


## Now training the model with the obtained tuned hyperparameters ##

In [43]:
# creating the model with the best hyperparameters
#model = tuner.hypermodel.build(best_hps)
#print(model.summary)
#print(model.optimizer.get_config())

model = creating_model_without_tuner()
# training
EPOCHS = 10
BATCH_SIZE = 4
# tensorboard logging dir
foldername = "/kaggle/working/tensorboardRecord"
os.makedirs(foldername, exist_ok=True)
logdir = os.path.join(foldername, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))


# modelsave checkpoint
modelDir = "/kaggle/working/modelDir"
os.makedirs(modelDir, exist_ok=True)
#filepath = 'my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
filepath = 'best_model.hdf5'
modelSavePath = os.path.join(modelDir, filepath)

tensorboardCallback = createTensorboardCallback(logdir)
modeSaveCallback = saveModelCallback(modelSavePath)
history = model.fit(train_dataset, epochs = EPOCHS, validation_data = validation_dataset, callbacks=[modeSaveCallback])


58900480/58889256 [==============================] - 2s 0us/step


2022-09-16 23:22:48.621418: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-09-16 23:22:48.621456: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-09-16 23:22:48.771920: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/10


2022-09-16 23:22:48.772084: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed


ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:835 run_step  **
        outputs = model.train_step(data)
    /opt/conda/lib/python3.7/site-pack

In [ ]:
import matplotlib.pyplot as plt
display(history)

# Evaluation on the test dataset #

In [ ]:
# loading the best saved model
inference_model = tf.keras.models.load_model("../input/ab-rsna-baseline-improve1/modelDir/best_model.hdf5")
inference_model.summary()
results = inference_model.evaluate(test_dataset, batch_size=16)
print("Evaluation on test data")
print(results)
